<a href="https://colab.research.google.com/github/willianrocha/bootcamp-datascience-alura/blob/main/module_5/ds_mod5_lecture1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

dados = pd.read_excel('https://github.com/willianrocha/bootcamp-datascience-alura/blob/main/files/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true')
dados.head()

PATIENT_VISIT_IDENTIFIER  AGE_ABOVE65  ...    WINDOW  ICU
0                         0            1  ...       0-2    0
1                         0            1  ...       2-4    0
2                         0            1  ...       4-6    0
3                         0            1  ...      6-12    0
4                         0            1  ...  ABOVE_12    1

[5 rows x 231 columns]

In [ ]:
def preenche_tabela(dados):
    features_continuas_colunas = dados.iloc[:, 13:-2].columns
    features_continuas = dados.groupby("PATIENT_VISIT_IDENTIFIER", as_index=False)[features_continuas_colunas].fillna(method='bfill').fillna(method='ffill')
    features_categoricas = dados.iloc[:, :13]
    saida = dados.iloc[:, -2:]
    dados_finais = pd.concat([features_categoricas, features_continuas, saida], ignore_index=True,axis=1)
    dados_finais.columns = dados.columns
    return dados_finais

In [ ]:
dados_limpos = preenche_tabela(dados)
a_remover = dados_limpos.query("WINDOW=='0-2' and ICU==1")['PATIENT_VISIT_IDENTIFIER'].values
dados_limpos = dados_limpos.query("PATIENT_VISIT_IDENTIFIER not in @a_remover")
dados_limpos = dados_limpos.dropna()
dados_limpos.describe()

PATIENT_VISIT_IDENTIFIER  ...          ICU
count               1760.000000  ...  1760.000000
mean                 192.818182  ...     0.201705
std                  110.637724  ...     0.401387
min                    0.000000  ...     0.000000
25%                   97.750000  ...     0.000000
50%                  191.500000  ...     0.000000
75%                  289.250000  ...     0.000000
max                  384.000000  ...     1.000000

[8 rows x 229 columns]

In [ ]:
def prepare_window(rows):
    if(np.any(rows["ICU"])):
        rows.loc[rows["WINDOW"]=="0-2", "ICU"] = 1
    return rows.loc[rows["WINDOW"] == "0-2"]

dados_limpos = dados_limpos.groupby("PATIENT_VISIT_IDENTIFIER").apply(prepare_window)
dados_limpos.AGE_PERCENTIL = dados_limpos.AGE_PERCENTIL.astype("category").cat.codes
dados_limpos.head()

PATIENT_VISIT_IDENTIFIER  AGE_ABOVE65  ...  WINDOW  ICU
PATIENT_VISIT_IDENTIFIER                                            ...             
0                        0                          0            1  ...     0-2    1
2                        10                         2            0  ...     0-2    1
3                        15                         3            0  ...     0-2    0
4                        20                         4            0  ...     0-2    0
5                        25                         5            0  ...     0-2    0

[5 rows x 231 columns]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

np.random.seed(73246)


x_columns = dados_limpos.columns
y = dados_limpos["ICU"]
x = dados_limpos[x_columns].drop(["ICU","WINDOW"], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y)
modelo = DummyClassifier()
modelo.fit(x_train, y_train)

y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)

/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


0.48863636363636365

In [ ]:
modelo = LogisticRegression(max_iter=10000)
modelo.fit(x_train, y_train)

y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)

0.6818181818181818

In [ ]:
for coluna in dados.columns:
    if coluna not in x_columns:
        print(coluna)

## Desafio 01: Transformar a coluna AGE_PERCENTIL em dados categóricos, mas utilizando outro método.

In [ ]:
dados_limpos = preenche_tabela(dados)
a_remover = dados_limpos.query("WINDOW=='0-2' and ICU==1")['PATIENT_VISIT_IDENTIFIER'].values
dados_limpos = dados_limpos.query("PATIENT_VISIT_IDENTIFIER not in @a_remover")
dados_limpos = dados_limpos.dropna()

dados_limpos = dados_limpos.groupby("PATIENT_VISIT_IDENTIFIER").apply(prepare_window)
# One Hot Encoding
dum_age = pd.get_dummies(dados_limpos['AGE_PERCENTIL'], prefix='AGE_PERCENTIL')
dados_limpos = dados_limpos.join(dum_age)
dados_limpos = dados_limpos.drop(columns=['AGE_PERCENTIL'])
dados_limpos.head()

PATIENT_VISIT_IDENTIFIER  ...  AGE_PERCENTIL_Above 90th
PATIENT_VISIT_IDENTIFIER                               ...                          
0                        0                          0  ...                         0
2                        10                         2  ...                         0
3                        15                         3  ...                         0
4                        20                         4  ...                         0
5                        25                         5  ...                         0

[5 rows x 240 columns]